In [1]:
import pandas as pd
import numpy as np
from datetime import time

In [3]:
df = pd.read_csv("dataset-3.csv")
df.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


# Python Task 2

### Question 1: Distance Matrix Calculation

Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [6]:
df.dtypes

id_start      int64
id_end        int64
distance    float64
dtype: object

In [8]:
def calculate_distance_matrix(dataset_path):
    distances = {}                                                               #Empyt dictionary
    for index, row in df.iterrows():
        start_location = row.id_start
        end_location = row.id_end
        distance = row.distance
        distances[(start_location, end_location)] = distance
        distances[(end_location, start_location)] = distance
    toll_locations = df.id_start.unique()
    distance_matrix = pd.DataFrame(0, index=toll_locations, columns=toll_locations)
    for i in toll_locations:
        for j in toll_locations:
            if i != j:
                direct_distance = distances.get((i, j), None)
                if direct_distance is not None:
                    distance_matrix.loc[i, j] = direct_distance
                else:
                    for k in toll_locations:
                        if i != k and j != k:
                            cumulative_distance = distance_matrix.loc[i, k] + distance_matrix.loc[k, j]
                            if distance_matrix.loc[i, j] == 0 or cumulative_distance < distance_matrix.loc[i, j]:
                                distance_matrix.loc[i, j] = cumulative_distance

    return distance_matrix

dataset = 'dataset-3.csv'
resulting_matrix = calculate_distance_matrix(dataset)
print(resulting_matrix)


C:\Users\prana\AppData\Local\Temp\ipykernel_6260\970981792.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '9.7' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  distance_matrix.loc[i, j] = direct_distance
C:\Users\prana\AppData\Local\Temp\ipykernel_6260\970981792.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '9.7' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  distance_matrix.loc[i, j] = cumulative_distance


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7      0.0      0.0      0.0      0.0      0.0   
1001402      9.7      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      0.0     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      0.0      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      0.0      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      0.0      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      0.0      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      0.0      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      0.0      0.0      0.0    

# Question 2: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [9]:
def unroll_distance_matrix(distance_matrix):
    upper_triangle = distance_matrix.where(np.triu(np.ones(distance_matrix.shape), k=1).astype(bool))
    unrolled_df = upper_triangle.stack().reset_index()
    unrolled_df.columns = ['id_start', 'id_end', 'distance']

    return unrolled_df

unrolled_df = unroll_distance_matrix(resulting_matrix)
print(unrolled_df)


     id_start   id_end  distance
0     1001400  1001402       9.7
1     1001400  1001404       0.0
2     1001400  1001406       0.0
3     1001400  1001408       0.0
4     1001400  1001410       0.0
..        ...      ...       ...
815   1001464  1001468       0.0
816   1001464  1001470       0.0
817   1001466  1001468      10.7
818   1001466  1001470       8.5
819   1001468  1001470      10.6

[820 rows x 3 columns]


# Question 3: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [13]:
def find_ids_within_ten_percentage_threshold(df, reference_value):
    reference_rows = df[df['id_start'] == reference_value]
    average_distance = reference_rows['distance'].mean()
    lower_threshold = average_distance - 0.1 * average_distance
    upper_threshold = average_distance + 0.1 * average_distance
    within_threshold_rows = df[(df['distance'] >= lower_threshold) & (df['distance'] <= upper_threshold)]
    result_ids = sorted(within_threshold_rows['id_start'].unique())

    return result_ids

reference_value = 1001468  # Taking last index value of id_start column as an integer
result_ids = find_ids_within_ten_percentage_threshold(unrolled_df, reference_value)
print(result_ids)


[1001400, 1001408, 1001420, 1001422, 1001438, 1001446, 1001450, 1001452, 1001466, 1001468]


# Question 4: Calculate Toll Rate
Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

0.8 for moto

1.2 for car

1.5 for rv

2.2 for bus

3.6 for truck

In [18]:
def calculate_toll_rate(df):
    #Parsing values for keys
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    for vehicle_type, rate_coefficient in rate_coefficients.items():
        df['vehicle_type'] = df.distance * rate_coefficient
    return df

result_with_toll_rates = calculate_toll_rate(unrolled_df)
print(result_with_toll_rates)


     id_start   id_end  distance  moto    car     rv    bus  truck  \
0     1001400  1001402       9.7  7.76  11.64  14.55  21.34  34.92   
1     1001400  1001404       0.0  0.00   0.00   0.00   0.00   0.00   
2     1001400  1001406       0.0  0.00   0.00   0.00   0.00   0.00   
3     1001400  1001408       0.0  0.00   0.00   0.00   0.00   0.00   
4     1001400  1001410       0.0  0.00   0.00   0.00   0.00   0.00   
..        ...      ...       ...   ...    ...    ...    ...    ...   
815   1001464  1001468       0.0  0.00   0.00   0.00   0.00   0.00   
816   1001464  1001470       0.0  0.00   0.00   0.00   0.00   0.00   
817   1001466  1001468      10.7  8.56  12.84  16.05  23.54  38.52   
818   1001466  1001470       8.5  6.80  10.20  12.75  18.70  30.60   
819   1001468  1001470      10.6  8.48  12.72  15.90  23.32  38.16   

     vehicle_type  
0           34.92  
1            0.00  
2            0.00  
3            0.00  
4            0.00  
..            ...  
815          0.00  

# Question 5: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

start_day, end_day must be strings with day values (from Monday to Sunday in proper case)

start_time and end_time must be of type datetime.time() with the values from time range given below.
                                                                                                                                         
Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

From 00:00:00 to 10:00:00: Apply a discount factor of 0.8

                                                                                                                                         
From 10:00:00 to 18:00:00: Apply a discount factor of 1.2

                                                                                                                                         
From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
                                                                                                                                         
Weekends (Saturday and Sunday):

Apply a constant discount factor of 0.7 for all times.
                                                                                                                                         
For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [20]:
def calculate_time_based_toll_rates(df):
    time_ranges_weekdays = [(time(0, 0, 0), time(10, 0, 0)),
                            (time(10, 0, 0), time(18, 0, 0)),
                            (time(18, 0, 0), time(23, 59, 59))]

    time_ranges_weekends = [(time(0, 0, 0), time(23, 59, 59))]
    df['start_day'] = df['end_day'] = df['start_time'] = df['end_time'] = None

    def map_time_range(start, end, time_ranges):
        for time_range in time_ranges:
            if start >= time_range[0] and end <= time_range[1]:
                return time_range
        return None

    def apply_time_based_rates(row, time_ranges, discount_factor):
        start_day = row['start_day']
        end_day = row['end_day']
        start_time = row['start_time']
        end_time = row['end_time']

        for time_range in time_ranges:
            if start_time >= time_range[0] and end_time <= time_range[1]:
                row['start_time'] = time_range[0]
                row['end_time'] = time_range[1]
                row['start_day'] = start_day
                row['end_day'] = end_day
                row[['moto', 'car', 'rv', 'bus', 'truck']] *= discount_factor
                return row

    for index, row in df.iterrows():
        start_day = row['start_day']
        end_day = row['end_day']

        if start_day == end_day:
            if start_day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']: 
                #we can also put if start_day in ['Saturday', 'Sunday']: --> else :
                row = apply_time_based_rates(row, time_ranges_weekdays, 0.8)
            elif start_day in ['Saturday', 'Sunday']:
                row = apply_time_based_rates(row, time_ranges_weekends, 0.7)

    return df

result_with_time_based_rates = calculate_time_based_toll_rates(result_with_toll_rates)
print(result_with_time_based_rates)

     id_start   id_end  distance  moto    car     rv    bus  truck  \
0     1001400  1001402       9.7  7.76  11.64  14.55  21.34  34.92   
1     1001400  1001404       0.0  0.00   0.00   0.00   0.00   0.00   
2     1001400  1001406       0.0  0.00   0.00   0.00   0.00   0.00   
3     1001400  1001408       0.0  0.00   0.00   0.00   0.00   0.00   
4     1001400  1001410       0.0  0.00   0.00   0.00   0.00   0.00   
..        ...      ...       ...   ...    ...    ...    ...    ...   
815   1001464  1001468       0.0  0.00   0.00   0.00   0.00   0.00   
816   1001464  1001470       0.0  0.00   0.00   0.00   0.00   0.00   
817   1001466  1001468      10.7  8.56  12.84  16.05  23.54  38.52   
818   1001466  1001470       8.5  6.80  10.20  12.75  18.70  30.60   
819   1001468  1001470      10.6  8.48  12.72  15.90  23.32  38.16   

     vehicle_type start_day end_day start_time end_time  
0           34.92      None    None       None     None  
1            0.00      None    None       N